In [17]:
from numpy import expand_dims
from numpy import mean
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from tensorflow.keras import backend
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Conv1D, Reshape,Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.constraints import Constraint
from matplotlib import pyplot
import pandas as pd
import numpy as np

In [19]:
# define wasserstein loss
def wasserstein_loss(y_true, y_pred):
    return backend.mean(y_true * y_pred)

In [20]:
def discriminator(in_shape=(2475,4)):
    model = Sequential()
    model.add(Conv1D(64, 10, strides=2, padding='same', input_shape=in_shape))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(Flatten())
    model.add(Dense(1))
    model.compile(loss=wasserstein_loss, optimizer='Adam')

    return model

In [21]:
def generator(in_shape=(30,30,3)):
    model = Sequential()   
    model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (3,3), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(Reshape((225,1,32)))
    model.add(Conv2DTranspose(32,(3,3),strides=(11,1),padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(Reshape((2475,32)))
    model.add(Conv1D(4, 5, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    return model
    
   
    


In [24]:
def gan(generator, discriminator):
    for layer in discriminator.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False
    model=Sequential() 
    model.add(generator)
    model.add(discriminator)
    model.compile(loss=wasserstein_loss, optimizer='Adam')
    return model


In [25]:
def load_reals():
    real=pd.read_csv('Producer_start_center_hetrogenous.csv')
    real=np.array(real[['WOPR1','WOPR2','WWPR1','WWPR2']].astype('float32'))
    X=np.zeros((400,2560,4))
    for i in range(400):
        X[i,:,:]=real[2560*i:2560*(1+i),:]
    
    return X
    

    


In [26]:
def generate_noise(n_samples):
    x_input = randn(30*30 * n_samples)
    x_input = x_input.reshape(n_samples,30,30)
    return x_input

In [27]:
def inputs():
    real=pd.read_csv('Producer_start_center_hetrogenous.csv')
    real=np.array(real[['X_loc','Y_loc']].astype('float32'))
    input=np.zeros((400,30,30,2))
    for i in range(400):
        x=int(real[2560*i,0]-1)
        input[i,x,:,0]=1
        y=int(real[2560*i,1]-1)
        input[i,:,y,1]=1  
    return input

In [28]:
def generate_inputs(input,noise,ix):
    
    input1=np.zeros((400,30,30,3))
       
    input1[:,:,:,0:2]=input
    input1=input1[ix]
    for i in range(len(ix)):
        input1[i,:,:,2]=noise[i,:,:]
    
    return input1
    

In [29]:
def generate_real_samples(dataset,ix,n_samples):
    X=dataset[ix]
    y=np.ones(n_samples)
    return X,y

In [36]:
def generate_fake_samples(generator,ix,n_samples,input):
    noise=generate_noise(n_samples)
    x_input = generate_inputs(input,noise,ix)
    X = generator.predict(x_input)
    y = np.zeros(n_samples)
    return X, y

In [39]:
def train(generator, discriminator, gan_model, dataset, n_epochs=10, n_samples=16,n_disc=5):
    bat_per_epo = int(dataset.shape[0] / n_samples)
    n_steps=bat_per_epo*n_epochs
    half_batch = int(n_samples / 2)
    disc1_hist, disc2_hist, gan_hist = list(), list(), list()
    for i in range(n_steps):
        disc1_tmp, disc2_tmp = list(), list()
        for j in range(n_disc):
            ix=randint(0, dataset.shape[0], half_batch)
            X_real, y_real = generate_real_samples(dataset,ix,half_batch)
            disc_loss1 = discriminator.train_on_batch(X_real, y_real)
            disc1_tmp.append(disc_loss1)
            X_fake, y_fake = generate_fake_samples(generator,ix,half_batch,input)
            disc_loss2 = discriminator.train_on_batch(X_fake, y_fake)
            disc1_tmp.append(disc_loss2)
        disc1_hist.append(mean(disc1_tmp))
        disc2_hist.append(mean(disc2_tmp))
        ix=randint(0, dataset.shape[0], n_samples)
        noise=generate_noise(n_samples)
        X_gan = generate_inputs(input,noise,ix)
        y_gan= np.zeros(n_samples)
        gan_loss = gan_model.train_on_batch(X_gan, y_gan)
        gan_hist.append(gan_loss)

        

In [31]:
dataset=load_reals()
input=inputs()
print(dataset.shape)
print(input.shape)
discriminator=discriminator(in_shape=(2475,4))
print('discriminator:')
discriminator.summary()
generator=generator(in_shape=(30,30,3))
print('generator:')
generator.summary()

(400, 2560, 4)
(400, 30, 30, 2)
discriminator:
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1238, 64)          2624      
_________________________________________________________________
batch_normalization_3 (Batch (None, 1238, 64)          256       
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 1238, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 79232)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 79233     
Total params: 82,113
Trainable params: 81,985
Non-trainable params: 128
_________________________________________________________________
generator:
Model: "sequential_3"
__________________________________

In [32]:
## make the dimensions same 
dataset=dataset[:,0:2475,:]
print(dataset.shape)

(400, 2475, 4)


In [33]:
gan_model=gan(generator, discriminator)


In [34]:
gan_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_3 (Sequential)    (None, 2475, 4)           30676     
_________________________________________________________________
sequential_2 (Sequential)    (None, 1)                 82113     
Total params: 112,789
Trainable params: 30,540
Non-trainable params: 82,249
_________________________________________________________________


In [ ]:
train(generator, discriminator, gan_model, dataset, n_epochs=10, n_samples=16,n_disc=5)

/Users/apple/miniconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/apple/miniconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
